In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1695750702828_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1695750702828_0002,pyspark,idle,Link,Link,None,✔


In [3]:
import pandas as pd
import numpy as np
import io
import os
import ipywidgets
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
PATH = 's3://oc-p8-xparisot'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results' 
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-p8-xparisot
PATH_Data:   s3://oc-p8-xparisot/Test
PATH_Result: s3://oc-p8-xparisot/Results

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7437|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7373|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7214|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7181|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7168|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7156|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  7101|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  6899|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:18|  6893|[FF D8 FF E0 00 1...|
|s3://oc-p8-xparis...|2023-08-26 08:22:15|  6586|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only s

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(10,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------+-----------+
|path                                              |label      |
+--------------------------------------------------+-----------+
|s3://oc-p8-xparisot/Test/Raspberry/176_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/174_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/268_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/302_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/315_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/309_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/27_100.jpg     |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/255_100.jpg    |Raspberry  |
|s3://oc-p8-xparisot/Test/Raspberry/292_100.jpg    |Raspberry  |
|s3://oc-p

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1695750702828_0002/container_1695750702828_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:403: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [14]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://oc-p8-xparisot/Results

In [17]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df.head(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://oc-p8-xparisot/Test/Cauliflower/r2_77_100...  ...  [0.0, 0.54935694, 0.32115462, 0.0, 0.09093891,...
1  s3://oc-p8-xparisot/Test/Cauliflower/r2_253_10...  ...  [0.0, 1.468065, 1.8280059, 0.0, 0.0, 0.0, 0.0,...
2  s3://oc-p8-xparisot/Test/Pepper Yellow/r_10_10...  ...  [0.5498254, 0.21695374, 0.0, 0.0, 0.53098583, ...
3          s3://oc-p8-xparisot/Test/Kaki/229_100.jpg  ...  [1.1535661, 0.3770244, 0.0, 0.0, 0.2686642, 1....
4     s3://oc-p8-xparisot/Test/Mulberry/r_25_100.jpg  ...  [0.31999612, 1.527524, 0.03644372, 0.07224511,...
5   s3://oc-p8-xparisot/Test/Tomato 2/r2_289_100.jpg  ...  [0.7105059, 0.0, 0.0, 0.09826345, 0.8885913, 0...
6     s3://oc-p8-xparisot/Test/Grape Blue/10_100.jpg  ...  [0.0, 0.0, 0.0, 0.123431526, 0.0, 0.87218374, ...
7   s3://oc-p8-xparisot/Test/Raspberry/r_251_100.jpg  ...  [0.0, 1.1963552, 0.07537578, 0.0, 0.23387216, ...
8    s3://oc-p8-xpa

In [20]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [22]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession

# Initialisation de Spark
spark = SparkSession.builder.appName("PCA Example").getOrCreate()

# Supposons que vous ayez un DataFrame Spark `features_df` avec une colonne "features" 
# contenant des listes qui doivent être converties en vecteurs denses.

# Convertir la colonne "array" en un vecteur dense
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
features_df = features_df.withColumn("features_vec", list_to_vector_udf("features"))

# Appliquer PCA
pca = PCA(k=2, inputCol="features_vec", outputCol="pcaFeatures")
model = pca.fit(features_df)
transformed = model.transform(features_df)

transformed.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+--------------------+--------------------+--------------------+
|                path|        label|            features|        features_vec|         pcaFeatures|
+--------------------+-------------+--------------------+--------------------+--------------------+
|s3://oc-p8-xparis...|  Apple Red 1|[0.8864111, 0.0, ...|[0.88641107082366...|[-5.9864706925665...|
|s3://oc-p8-xparis...|Pepper Yellow|[0.25667733, 0.38...|[0.25667732954025...|[-8.9269581810385...|
|s3://oc-p8-xparis...|         Kaki|[0.8176285, 0.036...|[0.81762850284576...|[-12.324092321756...|
|s3://oc-p8-xparis...|       Pepino|[0.28160787, 0.05...|[0.28160786628723...|[-8.8868142939590...|
|s3://oc-p8-xparis...|   Grape Blue|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.22...|[-7.7866972004778...|
+--------------------+-------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [23]:
from pyspark.ml.feature import PCA
import pandas as pd
import numpy as np

# 1. Effectuer une PCA complète
n = len(features_df.select("features_vec").first()[0])
pca = PCA(k=n, inputCol="features_vec", outputCol="pcaAllFeatures")
model = pca.fit(features_df)

# 2. Calculer la variance expliquée
explained_var = model.explainedVariance.toArray()

# 3. Calculer la variance cumulée
cumulative_explained_var = np.cumsum(explained_var)

# 4. Créer un DataFrame pour stocker les résultats
results_df = pd.DataFrame({
    'Component Number': range(1, n+1),
    'Explained Variance': explained_var,
    'Cumulative Explained Variance': cumulative_explained_var
})

# 5. Identifier le nombre de composantes nécessaires pour atteindre 90% et 100% de la variance cumulée
num_components_90 = np.where(cumulative_explained_var >= 0.9)[0][0] + 1
num_components_100 = n

# 6. Ajouter des informations sur le nombre de composantes pour 90% et 100% de la variance cumulée
results_df.loc[num_components_90 - 1, 'Note'] = '90% of variance is explained by this component'
results_df.loc[num_components_100 - 1, 'Note'] = '100% of variance is explained by this component'

# 7. Afficher le DataFrame avec les résultats
print(results_df)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      Component Number  ...                                             Note
0                    1  ...                                              NaN
1                    2  ...                                              NaN
2                    3  ...                                              NaN
3                    4  ...                                              NaN
4                    5  ...                                              NaN
...                ...  ...                                              ...
1275              1276  ...                                              NaN
1276              1277  ...                                              NaN
1277              1278  ...                                              NaN
1278              1279  ...                                              NaN
1279              1280  ...  100% of variance is explained by this component

[1280 rows x 4 columns]

In [24]:
features_df.select("path", "label", "features_vec")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, label: string, features_vec: vector]

In [25]:
features_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+--------------------+--------------------+
|                path|        label|            features|        features_vec|
+--------------------+-------------+--------------------+--------------------+
|s3://oc-p8-xparis...|  Apple Red 1|[0.8864111, 0.0, ...|[0.88641107082366...|
|s3://oc-p8-xparis...|Pepper Yellow|[0.25667733, 0.38...|[0.25667732954025...|
|s3://oc-p8-xparis...|         Kaki|[0.8176285, 0.036...|[0.81762850284576...|
|s3://oc-p8-xparis...|     Tomato 2|[0.7105059, 0.0, ...|[0.71050590276718...|
|s3://oc-p8-xparis...|   Grape Blue|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.12...|
|s3://oc-p8-xparis...|   Grape Blue|[0.0, 0.0, 0.0, 0...|[0.0,0.0,0.0,0.22...|
|s3://oc-p8-xparis...|    Raspberry|[0.0, 1.1963552, ...|[0.0,1.1963552236...|
|s3://oc-p8-xparis...|     Tomato 2|[0.48791632, 1.80...|[0.48791632056236...|
|s3://oc-p8-xparis...|         Kaki|[0.8992836, 0.077...|[0.89928358793258...|
|s3://oc-p8-xparis...|  Cauliflower|[0.0, 0.54935694

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws

# Assurez-vous d'avoir une session Spark active
spark = SparkSession.builder.appName("DirectJSONSave").getOrCreate()

# Convertir la colonne "features" en une chaîne
features_df = features_df.withColumn("features_str", concat_ws(",", "features"))

# Écrire le DataFrame au format JSON
output_path_json = "s3://oc-p8-xparisot/original_features.json"
features_df.coalesce(1).write.mode("overwrite").json(output_path_json)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
transformed.select("path", "label", "pcaFeatures")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, label: string, pcaFeatures: vector]

In [28]:
transformed.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+--------------------+--------------------+--------------------+
|                path|        label|            features|        features_vec|         pcaFeatures|
+--------------------+-------------+--------------------+--------------------+--------------------+
|s3://oc-p8-xparis...|  Apple Red 1|[0.8864111, 0.0, ...|[0.88641107082366...|[-5.9864706925665...|
|s3://oc-p8-xparis...|Pepper Yellow|[0.25667733, 0.38...|[0.25667732954025...|[-8.9269581810385...|
|s3://oc-p8-xparis...|         Kaki|[0.8176285, 0.036...|[0.81762850284576...|[-12.324092321756...|
|s3://oc-p8-xparis...|  Cauliflower|[0.0, 0.54935694,...|[0.0,0.5493569374...|[5.15698464650827...|
|s3://oc-p8-xparis...|  Cauliflower|[0.0, 1.468065, 1...|[0.0,1.4680650234...|[5.87046633771919...|
|s3://oc-p8-xparis...|    Raspberry|[0.0, 1.1963552, ...|[0.0,1.1963552236...|[3.84648640978732...|
|s3://oc-p8-xparis...|Pepper Yellow|[0.5498254, 0.216...|[0.54982537031173...|[-9.1694598979496...|


In [29]:
from pyspark.sql.functions import concat_ws

# Convertir la colonne "features" en une chaîne
transformed = transformed.withColumn("features_str", concat_ws(",", "features"))

# Écrire le DataFrame au format JSON
output_path_json = "s3://oc-p8-xparisot/transformed_features.json"
transformed.coalesce(1).write.mode("overwrite").json(output_path_json)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Chargement des données enregistrées et validation du résultat

In [30]:
df_wk = spark.read.parquet(PATH_Result)
df_wk = df_wk.toPandas()
df_wk.head(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://oc-p8-xparisot/Test/Cauliflower/r_91_100.jpg  ...  [0.0, 0.8385670185089111, 1.8494701385498047, ...
1   s3://oc-p8-xparisot/Test/Raspberry/r_139_100.jpg  ...  [0.07384958863258362, 0.1792396754026413, 0.0,...
2         s3://oc-p8-xparisot/Test/Kaki/r_26_100.jpg  ...  [2.1866676807403564, 0.025079533457756042, 0.0...
3          s3://oc-p8-xparisot/Test/Kaki/r_1_100.jpg  ...  [0.5301712155342102, 0.35785001516342163, 0.0,...
4  s3://oc-p8-xparisot/Test/Apple Red 1/r_197_100...  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.8592246770858765, ...
5  s3://oc-p8-xparisot/Test/Tomato Maroon/183_100...  ...  [0.0, 0.8248069882392883, 0.0, 0.0, 0.0, 1.368...
6  s3://oc-p8-xparisot/Test/Tomato Maroon/178_100...  ...  [0.0, 0.8894882798194885, 0.0, 0.0, 0.0, 0.972...
7       s3://oc-p8-xparisot/Test/Tomato 2/57_100.jpg  ...  [0.10844150930643082, 0.04094743728637695, 0.0...
8        s3://oc-p8